In [ ]:
import dask.dataframe as dd
from dask.multiprocessing import get
import ast,os
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt

In [ ]:
### Combines all the chunks of csv files generated from the monte carlo simulations. 
### Over here is the example of NE Region of Bangladesh 
directory_path = r'Path to Directory where Monte Carlo Simulations will be saved/NE/'
dfs = []

for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path)
        dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)

In [ ]:
def convert_string_to_list(str_tuple):
    if pd.notna(str_tuple):
        tuple_result = ast.literal_eval(str_tuple)
        return list(tuple_result)
    else:
        return []

# read the CSV file using Dask DataFrame
ddf = dd.concat(dfs, ignore_index=True)

# apply your function
meta = pd.Series(dtype=object)  # adjust this to match the output type of your function
ddf['Gauge_Combination'] = ddf['Gauge_Combination'].map(convert_string_to_list, meta=meta)

# select your desired columns
ddf = ddf[['Gauge_Combination','Cummulative_Volume']]

# compute the result
df_result = ddf.compute().reset_index(drop = True)
# Calculate the length of each 'Names' list
df_result['Names_length'] = df_result['Gauge_Combination'].apply(len)
df_result

df_result.head(3)


In [ ]:
Benchmark_Vol = df_result['Cummulative_Volume'][df_result['Names_length'].idxmax()]
df_result1 = df_result.copy()
# df_result1 = df_result1.drop(df_result1[df_result1['Names_length'] == 14].index)
df_result1['PError'] = (abs(df_result1['Cummulative_Volume'] - Benchmark_Vol))/Benchmark_Vol
df_result1=df_result1[['Gauge_Combination','PError']]
# Define a threshold for 'high performing' subsets
threshold = 0.1

# Count occurrences of gauges in high performing subsets
gauge_counter = Counter()
df_tuple = tuple(df_result1.to_records(index=False))
for subset, score in df_tuple:
    if score <= threshold:
        gauge_counter.update(subset)

# Print gauges sorted by their count (most common first)
print(gauge_counter.most_common())
